In [47]:
import pandas as pd
import numpy as np
import math
import os
from matplotlib import pyplot as plt
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run UtilsLightGBMFingerPrinting.ipynb

In [48]:
FILES = [
    
    #k = 1 SVR
    './FinalResult/Resultados_Metodo_13.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_14.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_15.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 2 SVR
    './FinalResult/Resultados_Metodo_16.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_17.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_18.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 3 SVR
    './FinalResult/Resultados_Metodo_19.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_20.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_21.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 4 SVR
    './FinalResult/Resultados_Metodo_22.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_23.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_24.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 5 SVR
    './FinalResult/Resultados_Metodo_25.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_26.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_27.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    
]
METHODS = [x.split("_")[-1].split(".")[0] for x in FILES]     # lista contendo os metodos que serão usados
db = pd.read_csv('databases/meds_pandas.csv')
erbs = pd.read_csv('databases/Bts.csv')
#errorLocation = pd.read_csv('result/erros.csv')
ERROS_CSV_FILE = './result-k-fold/erros-SVR-k-fold.csv' 

In [52]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,48.46,76.55,63.94,50.56,77.21,64.91,51.85,92.15,69.58,47.85,73.79,60.25,50.31,76.74,60.66
std,41.75,37.92,42.31,42.47,38.32,46.02,42.92,23.84,40.91,42.56,37.22,40.63,40.16,38.12,42.05
min,0.90,17.44,1.91,1.87,17.44,2.00,0.44,32.13,1.24,0.74,17.44,1.61,0.94,17.44,0.65
25%,15.37,42.13,26.67,17.81,42.13,24.38,18.62,81.69,32.13,15.33,42.13,23.67,17.13,42.13,22.56
50%,34.20,87.04,64.27,35.61,87.04,64.27,38.94,87.04,72.88,37.14,68.72,61.58,38.33,87.04,55.04
75%,72.01,102.64,97.20,74.11,102.64,97.92,76.48,102.64,102.64,68.64,102.64,87.04,75.59,102.64,96.42
max,187.26,137.37,225.27,224.26,137.37,258.61,237.98,137.37,206.63,249.75,137.37,191.02,180.03,137.37,196.48


# SVR-Outdoor:

In [62]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df
mean_outdoor = (test_erros['13'] + test_erros['16'] + test_erros['19'] + test_erros['22'] + test_erros['25'])/5
mean_outdoor.head()

0    42.500
1    69.042
2    39.062
3    91.924
4    53.872
dtype: float64

In [63]:
min_outdoor = min(test_erro['13']['min'], test_erro['16']['min'], test_erro['19']['min'], test_erro['22']['min'], test_erro['25']['min'])
min_outdoor

0.44

In [64]:
max_outdoor = max(test_erro['13']['max'], test_erro['16']['max'], test_erro['19']['max'], test_erro['22']['max'], test_erro['25']['max'])
max_outdoor

249.75

# SVR-Indoor:

In [65]:
mean_indoor = (test_erros['14'] + test_erros['17'] + test_erros['20'] + test_erros['23'] + test_erros['26'])/5
mean_indoor.head()

0    28.528
1    42.130
2    28.528
3    96.420
4    50.318
dtype: float64

In [66]:
min_outdoor = min(test_erro['14']['min'], test_erro['17']['min'], test_erro['20']['min'], test_erro['23']['min'], test_erro['26']['min'])
min_outdoor

17.44

In [67]:
max_outdoor = max(test_erro['14']['max'], test_erro['17']['max'], test_erro['20']['max'], test_erro['23']['max'], test_erro['26']['max'])
max_outdoor

137.37

# SVR-Indoor-Outdoor:

In [68]:
mean_indoor_outdoor = (test_erros['15'] + test_erros['18'] + test_erros['21'] + test_erros['24'] + test_erros['27'])/5
mean_indoor_outdoor.head()

0    70.468
1    48.744
2    34.718
3    26.106
4    65.814
dtype: float64

In [69]:
min_outdoor = min(test_erro['15']['min'], test_erro['18']['min'], test_erro['21']['min'], test_erro['24']['min'], test_erro['27']['min'])
min_outdoor

0.65

In [70]:
max_outdoor = max(test_erro['15']['max'], test_erro['18']['max'], test_erro['21']['max'], test_erro['24']['max'], test_erro['27']['max'])
max_outdoor

258.61

In [44]:
#errors_df.to_csv(ERROS_CSV_FILE, index=False)

In [75]:
erros_SVR = [mean_outdoor,mean_indoor, mean_indoor_outdoor ]
erros_SVR = pd.DataFrame(erros_SVR)
erros_SVR = erros_SVR.T
erros_SVR = round(erros_SVR,2)
erros_SVR.to_csv(ERROS_CSV_FILE, index=False)
erros_SVR.head()


,0,1,2
0,42.50,28.53,70.47
1,69.04,42.13,48.74
2,39.06,28.53,34.72
3,91.92,96.42,26.11
4,53.87,50.32,65.81


In [78]:
results_SVR = round(erros_SVR.describe(), 2)
results_SVR
#results_SVR.to_csv('./result-k-fold/Result-SVR-k-fold.csv', index=False)

,0,1,2
count,400.00,400.00,400.00
mean,49.80,79.29,63.87
std,18.32,34.10,29.72
min,12.15,28.53,14.42
25%,36.56,42.13,41.44
50%,49.27,87.04,61.09
75%,61.06,102.64,87.04
max,108.21,137.37,137.37
